**plotly_live_update_example_dash.ipynb**

Wonhee Lee

2024 MAY 10 (FRI)

reference:

- https://velog.io/@sn50hee/3%EC%A3%BC%EC%B0%A8-Dash-Live-Updating-Components
- https://www.tutorialspoint.com/plot-live-graphs-using-python-dash-and-plotly
- https://plotly.com/python/creating-and-updating-figures/
- https://dash.plotly.com/tutorial
- https://abluesnake.tistory.com/152

In [1]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import dash
from dash import Dash, dcc, html, Input, Output

In [3]:
import numpy as np

# data

In [4]:
def model(theta, x):
    w, b = theta
    return w*x + b

In [5]:
theta = (3, 1)

In [6]:
x = np.arange(7)

noise = np.random.normal(scale=4.5, size=(7,))
y = model(theta, x) + noise

print(x)
print(y)

[0 1 2 3 4 5 6]
[-0.9288641   0.41805304  5.01472681 13.73163016 14.80579838 17.28171209
 20.12490495]


# figure

In [7]:
fig = px.scatter(x=x, y=y)
fig_model = px.line(x=x, y=model(theta, x))
fig_model.data[0].line.color = "#e02a19"
fig.add_trace(fig_model.data[0])

fig.show()

# static Dash app test

In [8]:
# initialize Dash app
app = Dash(__name__, external_stylesheets=None)

# define app layout
app.layout = html.Div([
    #
    html.H3("My Live Plot"),
    #
    dcc.Graph(id="my_live_plot", figure=fig),
    #
    dcc.Interval(
        id="interval_manager",
        interval=1000,  # [ms] callback interval
        n_intervals=0)])  # interval count

In [9]:
# # run app
# if __name__ == '__main__':
#     app.run(debug=True)

# dynamic Dash app test

In [10]:
# initialize Dash app
app = Dash(__name__)

# define app layout
app.layout = html.Div([
    #
    html.H3("My Live Plot"),
    #
    dcc.Graph(id="my_live_plot"),
    #
    dcc.Interval(
        id="interval_manager",
        interval=1000,  # [ms] callback interval
        n_intervals=0)])  # interval count

In [11]:
# callback function
@app.callback(Output(component_id="my_live_plot", component_property="figure"),
    Input(component_id="interval_manager", component_property="n_intervals"))
def update_figure(n):
    global y
    fig = px.scatter(x=x, y=y)
    fig_model = px.line(x=x, y=model(theta, x))
    fig_model.data[0].line.color = "#e02a19"
    fig.add_trace(fig_model.data[0])
    return fig

In [12]:
# run app
if __name__ == '__main__':
    app.run(debug=True, port=8051)

# the live plot will be shown below, but it can also be monitored on a web page: http://localhost:8051/

# update loop

In [13]:
import time

In [14]:
for i in range(10):
    theta = (3 + np.sin(i), 1)
    noise = np.random.normal(scale=4.5, size=(7,))
    y = model(theta, x) + noise
    time.sleep(1)